## 1. Load the dataset



To run this notebook, you will need to install: pandas, openai, transformers, plotly, matplotlib, scikit-learn, torch (transformer dep), torchvision, and scipy.

In [17]:
# install openai (do this only once at the start. Source: https://community.openai.com/t/no-module-named-openai/8303/3)
!python --version
!pip --version
!pip install openai

In [10]:
import pandas as pd
import time
import openai
import json
import numpy as np
from openai.embeddings_utils import get_embedding
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [37]:
# Setup openAI and constants
openai.api_key = 
embedding_model = "text-embedding-ada-002"
select_query = "SELECT * FROM 'mortgage complaints'"
db = "sqlite:///StaterData.db"

# Retrieve the data from the database
input_datapath = pd.read_sql_query(select_query, db)

# Limit test size due to performance issues
data = input_datapath.loc[:200]

# Create and return embedding
def get_embedding(text, model="text-embedding-ada-002"):
    # sleep 1 second to prevent reaching rate limit. Limit: 60 requests per min.
    time.sleep(1)
    text = text.replace("\n", " ")
    return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

data['Embedding'] = data['Consumer complaint narrative'].apply(lambda x: get_embedding(x))
data.to_csv("StaterDataEmbeddings.csv", index=False)

In [21]:
data = pd.read_csv("StaterDataEmbeddings.csv")
data.head()

,index,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,Embedding
0,2,2023-03-16 00:00:00.000000,Mortgage,Conventional home mortgage,Trouble during payment process,None,"I closed on my house XX/XX/XXXX, XXXX payment...",None,FIFTH THIRD FINANCIAL CORPORATION,TX,78744,None,1,Web,2023-03-16 00:00:00.000000,Closed with non-monetary relief,1,1,6697855,"[-0.03669096902012825, -0.0033018486574292183,..."
1,5,2022-08-01 00:00:00.000000,Mortgage,VA mortgage,Applying for a mortgage or refinancing an exis...,None,XXXX and XXXX Mortgage payments were marked as...,Company has responded to the consumer and the ...,Freedom Mortgage Company,FL,33032,Servicemember,1,Web,2022-08-01 00:00:00.000000,Closed with explanation,1,1,5832311,"[-0.03389027342200279, -0.011986914090812206, ..."
2,6,2018-01-19 00:00:00.000000,Mortgage,Conventional home mortgage,Trouble during payment process,None,"XXXX XXXX XXXX On XX/XX/XXXX, I called 21st mo...",None,21ST MORTGAGE CORP.,FL,320XX,None,1,Web,2018-01-19 00:00:00.000000,Closed with explanation,1,1,2787647,"[-0.03640284389257431, -0.011248917318880558, ..."
3,9,2023-02-18 00:00:00.000000,Mortgage,FHA mortgage,Trouble during payment process,None,Loancare through XXXX XXXX XXXX' is negligentl...,Company has responded to the consumer and the ...,"LoanCare, LLC",FL,331XX,Servicemember,1,Web,2023-02-18 00:00:00.000000,Closed with explanation,1,1,6589323,"[-0.023640956729650497, -0.0031948164105415344..."
4,10,2023-03-01 00:00:00.000000,Mortgage,Other type of mortgage,Struggling to pay mortgage,None,"In XXXX of XXXX, I received a Loan Modificatio...",Company believes complaint caused principally ...,NATIONSTAR MORTGAGE LLC,NY,10512,None,1,Web,2023-03-01 00:00:00.000000,Closed with explanation,1,1,6633594,"[-0.025865409523248672, -0.003657393157482147,..."


In [36]:
# Prepare the feature matrix X and target vector y
X = data["Embedding"].apply(lambda x: json.loads(x)).tolist()
y = data["Issue"]

# Reshape the embeddings into a 2D array
X = np.array(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Create an instance of the random forest classifier
rf_classifier = RandomForestClassifier()

# Train the random forest classifier
rf_classifier.fit(X_train, y_train)

# Make predictions on the test data
y_pred = rf_classifier.predict(X_test)

# Evaluate the performance of the model
accuracy = accuracy_score(y_test, y_pred)
# accuracy = (y_pred == y_test).mean()
print("Accuracy:", accuracy)

Accuracy: 0.5409836065573771


Accuracy for this model is 0.49. It is ok as the complaints are not yet cleaned and have not used the best parameters according the gridsearch.<br>
Without random_state the accuracy varies between 0.49 and 0.62. If the model has random_state=2 as parameter, the accuracy is 0.49.